## Скачиваем данные и устанавливаем программы

In [ ]:
!wget https://raw.githubusercontent.com/yliava/hse22_hw2/main/data/gms2.lst

--2022-10-18 19:09:31--  https://raw.githubusercontent.com/yliava/hse22_hw2/main/data/gms2.lst
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220925 (216K) [text/plain]
Saving to: ‘gms2.lst.1’

gms2.lst.1          100%[===================>] 215.75K  --.-KB/s    in 0.02s   

2022-10-18 19:09:31 (11.6 MB/s) - ‘gms2.lst.1’ saved [220925/220925]



In [ ]:
!wget https://raw.githubusercontent.com/yliava/hse22_hw2/main/data/proteins.fasta

--2022-10-18 19:12:00--  https://raw.githubusercontent.com/yliava/hse22_hw2/main/data/proteins.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1404610 (1.3M) [text/plain]
Saving to: ‘proteins.fasta’

proteins.fasta      100%[===================>]   1.34M  --.-KB/s    in 0.04s   

2022-10-18 19:12:01 (37.9 MB/s) - ‘proteins.fasta’ saved [1404610/1404610]



In [ ]:
!wget https://raw.githubusercontent.com/yliava/hse22_hw2/main/data/scaffolds.hits_from_MIL_1.txt

--2022-10-18 19:09:31--  https://raw.githubusercontent.com/yliava/hse22_hw2/main/data/scaffolds.hits_from_MIL_1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 836180 (817K) [text/plain]
Saving to: ‘scaffolds.hits_from_MIL_1.txt.1’

scaffolds.hits_from 100%[===================>] 816.58K  --.-KB/s    in 0.03s   

2022-10-18 19:09:32 (24.5 MB/s) - ‘scaffolds.hits_from_MIL_1.txt.1’ saved [836180/836180]



In [ ]:
!wget https://raw.githubusercontent.com/yliava/hse22_hw2/main/data/scaffolds.hits_from_SwissProt.txt

--2022-10-18 19:09:32--  https://raw.githubusercontent.com/yliava/hse22_hw2/main/data/scaffolds.hits_from_SwissProt.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66183 (65K) [text/plain]
Saving to: ‘scaffolds.hits_from_SwissProt.txt.1’

scaffolds.hits_from 100%[===================>]  64.63K  --.-KB/s    in 0.007s  

2022-10-18 19:09:32 (8.73 MB/s) - ‘scaffolds.hits_from_SwissProt.txt.1’ saved [66183/66183]



In [ ]:
!wget https://raw.githubusercontent.com/yliava/hse22_hw1/main/data/Poil_gapClosed.fa

--2022-10-18 19:09:32--  https://raw.githubusercontent.com/yliava/hse22_hw1/main/data/Poil_gapClosed.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3977716 (3.8M) [text/plain]
Saving to: ‘Poil_gapClosed.fa.1’

Poil_gapClosed.fa.1 100%[===================>]   3.79M  --.-KB/s    in 0.05s   

2022-10-18 19:09:32 (69.5 MB/s) - ‘Poil_gapClosed.fa.1’ saved [3977716/3977716]



## Скачиваем геном близкородственной бактерии T.oleivorans

In [ ]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [ ]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Data import CodonTable
from Bio import Entrez
from Bio.SeqFeature import SeqFeature, FeatureLocation
from datetime import datetime as dt
import pandas as pd

In [ ]:
!head -n 40 gms2.lst

##Аннотация генома

In [ ]:
scaffolds = dict()
for record in SeqIO.parse("/content/Poil_gapClosed.fa", "fasta"):
  record.annotations['molecule_type'] = 'DNA'
  record.annotations['date'] = dt.now().strftime("%d-%b-%Y").upper()
  record.annotations['data_file_division'] = 'BCT'
  
  scaffolds[record.id] = record

In [ ]:
genes = dict()
for gene in SeqIO.parse("proteins.fasta", "fasta"):
  desc = gene.description.split(' ')
  scaffold = desc[1]
  start, end = int(desc[2]), int(desc[3])
  strand = 1 if desc[4] == '+' else -1
  
  feat = SeqFeature(FeatureLocation(start, end, strand=strand), type="CDS")
  feat.qualifiers['locus_tag'] = [desc[0]]
  feat.qualifiers['translation'] = [gene.seq]
  scaffolds[scaffold].features.append(feat)
  
  genes[desc[0]] = feat


In [ ]:
feat.qualifiers

OrderedDict([('locus_tag', ['3622']),
             ('translation',
              [Seq('GLPLYRMETIFKRMGIHLPRNTLANWMIKSSECLQPLYNLLNDQLLESGYLHMD...DDY')])])

## Добавление функций белков из MIL_1 и БД (SwissProt)

In [ ]:
mil_1_genes = dict()
for feat in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
  if 'protein_id' not in feat.qualifiers:
    continue
  if 'product' not in feat.qualifiers:
    continue
  
  mil_1_genes[feat.qualifiers['protein_id'][0]] = feat.qualifiers['product'][0]

In [ ]:
names = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
mil_1_hits = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=names)
mil_1_hits

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,2,lcl|HF680312.1_prot_CCU71928.1_1479,100.000,173,0,0,1,173,1,173,1.660000e-130,359.0
1,4,lcl|HF680312.1_prot_CCU71924.1_1475,97.143,35,1,0,1,35,1,35,1.510000e-19,75.1
2,5,lcl|HF680312.1_prot_CCU71587.1_1138,98.770,244,3,0,1,244,38,281,2.170000e-171,470.0
3,5,lcl|HF680312.1_prot_CCU72192.1_1743,41.634,257,132,7,1,244,38,289,1.720000e-47,155.0
4,6,lcl|HF680312.1_prot_CCU71586.1_1137,100.000,391,0,0,1,391,1,391,0.000000e+00,810.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10083,3575,lcl|HF680312.1_prot_CCU73517.1_3068,96.552,116,4,0,1,116,216,331,4.090000e-80,241.0
10084,3575,lcl|HF680312.1_prot_CCU73713.1_3264,96.552,116,4,0,1,116,216,331,5.770000e-80,241.0
10085,3575,lcl|HF680312.1_prot_CCU73876.1_3427,96.552,116,4,0,1,116,223,338,8.800000e-80,241.0
10086,3575,lcl|HF680312.1_prot_CCU73197.1_2748,96.552,116,4,0,1,116,223,338,8.800000e-80,241.0


In [ ]:
hits = mil_1_hits[mil_1_hits['sseqid'].str.contains("CCU")].sort_values('bitscore', ascending=False).drop_duplicates('qseqid')
hits

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
2746,1016,lcl|HF680312.1_prot_CCU73896.1_3447,98.979,2253,23,0,1,2253,1,2253,0.000000e+00,4296.0
4094,1471,lcl|HF680312.1_prot_CCU70724.1_275,97.870,2113,43,1,7,2119,1,2111,0.000000e+00,4158.0
8685,3170,lcl|HF680312.1_prot_CCU71900.1_1451,99.878,1640,2,0,1,1640,1,1640,0.000000e+00,3406.0
2573,945,lcl|HF680312.1_prot_CCU73861.1_3412,99.214,1654,13,0,1,1654,1,1654,0.000000e+00,3372.0
9481,3447,lcl|HF680312.1_prot_CCU71621.1_1172,99.690,1611,5,0,1,1611,1,1611,0.000000e+00,3338.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10038,3562,lcl|HF680312.1_prot_CCU70835.1_386,59.259,54,21,1,2,55,2784,2836,1.630000e-12,56.6
9471,3439,lcl|HF680312.1_prot_CCU71629.1_1180,61.224,49,8,1,2,39,139,187,1.280000e-11,54.7
8765,3213,lcl|HF680312.1_prot_CCU73367.1_2918,41.096,73,43,0,5,77,8,80,7.960000e-13,54.7
9852,3533,lcl|HF680312.1_prot_CCU73711.1_3262,29.592,98,63,2,16,112,4,96,1.250000e-11,53.5


In [ ]:
for i, hit in hits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('_')[2]
  gene.qualifiers['product'] = [mil_1_genes[match]]

In [ ]:
!wget -nc https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
!gzip -d uniprot_sprot.dat.gz

--2022-10-18 19:27:26--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642093634 (612M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 612.35M  43.5MB/s    in 14s     

2022-10-18 19:27:40 (44.1 MB/s) - ‘uniprot_sprot.dat.gz’ saved [642093634/642093634]



In [ ]:
!grep '^ID\|^DE   RecName: Full=' uniprot_sprot.dat > SwissProt_names.txt

In [ ]:
previd = None
swissgenes = dict()
for line in open('SwissProt_names.txt'):
  if line.startswith('ID'):
    previd = line.split()[1]
  if line.startswith('DE'):
    swissgenes[previd] = line.split('=')[1][:-2]

In [ ]:
swisshits_raw = pd.read_csv('scaffolds.hits_from_SwissProt.txt', sep='\t', header=None, names=names)
swisshits = swisshits_raw.sort_values('bitscore', ascending=False).drop_duplicates('qseqid')

for i, hit in swisshits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('|')[-1]
  gene.qualifiers['product'] = [swissgenes[match]]

In [ ]:
SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

74